In [1]:
import nltk
import pickle
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
pTrees = nltk.corpus.treebank.parsed_sents()
%time len(pTrees) #hm... so counting through all of the trees takes about 5 seconds, but this isn't even touching any part of the tree.
print(pTrees[0])

Wall time: 4.53 s


3914

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [11]:
pTreeProductions = []
for tree in pTrees:
    pTreeProductions.append(tree.productions())

In [13]:
len(pTreeProductions)
pTreeProductions[0]

3914

[S -> NP-SBJ VP .,
 NP-SBJ -> NP , ADJP ,,
 NP -> NNP NNP,
 NNP -> 'Pierre',
 NNP -> 'Vinken',
 , -> ',',
 ADJP -> NP JJ,
 NP -> CD NNS,
 CD -> '61',
 NNS -> 'years',
 JJ -> 'old',
 , -> ',',
 VP -> MD VP,
 MD -> 'will',
 VP -> VB NP PP-CLR NP-TMP,
 VB -> 'join',
 NP -> DT NN,
 DT -> 'the',
 NN -> 'board',
 PP-CLR -> IN NP,
 IN -> 'as',
 NP -> DT JJ NN,
 DT -> 'a',
 JJ -> 'nonexecutive',
 NN -> 'director',
 NP-TMP -> NNP CD,
 NNP -> 'Nov.',
 CD -> '29',
 . -> '.']

In [39]:
def getAllInstances(wordType):
    instances = []
    for tree in pTreeProductions:
        for rule in tree:
            text = str(rule)
            left, right = text.split(" -> ")
            #print("Left, Rule:")
            #print(left)
            #print(rule)
            #print()
            if(left == wordType):
                instances.append(rule)
    return instances

In [42]:
adjs = getAllInstances('JJ')
len(adjs)

5834